In [2]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import pickle
import cv2
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout



In [3]:
baseDir=os.getcwd()
dataDir=os.path.join(baseDir, 'data')
training_file=os.path.join(dataDir,'train.p')
testing_file=os.path.join(dataDir,'test.p')

In [4]:
with open(training_file,'rb') as f:
    train = pickle.load(f)
    
with open(testing_file,'rb') as f:
    test = pickle.load(f)

In [5]:
train.keys()

dict_keys(['sizes', 'coords', 'features', 'labels'])

In [6]:
test.keys()

dict_keys(['sizes', 'coords', 'features', 'labels'])

In [7]:
train['labels']

array([ 0,  0,  0, ..., 42, 42, 42], dtype=uint8)

In [8]:
X_train,y_train=train['features'],train['labels']
X_test,y_test=test['features'],test['labels']

In [8]:
X_train[0].shape

(32, 32, 3)

In [9]:
print('Image Shape:    {}'.format(X_train[0].shape))
print('Training Set:   {} samples'.format(len(X_train)))
print('Test Set:       {} samples'.format(len(X_test)))
##print('Labels in Set:  {}'.format(np.unique(y_train)))
print('Number of unique classes: {}'.format(len(np.unique(y_train))))

Image Shape:    (32, 32, 3)
Training Set:   39209 samples
Test Set:       12630 samples
Number of unique classes: 43


In [10]:
signnames=pd.read_csv('signnames.csv')

In [11]:
from sklearn.utils import shuffle
X_train, y_train=shuffle(X_train, y_train)

In [12]:
from sklearn.model_selection import train_test_split
test_features, valid_features, test_labels, valid_labels= train_test_split(X_test,y_test, test_size=0.60, random_state=42)

In [13]:
print('Number of validation examples = {}'.format(len(valid_labels)))
print('Number of testing examples = {}'.format(len(test_labels)))

Number of validation examples = 7578
Number of testing examples = 5052


In [14]:
print("X_train.shape: {}".format(X_train.shape))
print("valid_features.shape: {}".format(valid_features.shape))
print("y_train.shape: {}".format(y_train.shape))
print("valid_labels.shape: {}".format(valid_labels.shape))

X_train.shape: (39209, 32, 32, 3)
valid_features.shape: (7578, 32, 32, 3)
y_train.shape: (39209,)
valid_labels.shape: (7578,)


### One hot encoding the labels

In [15]:
from keras.utils import to_categorical
y_train = to_categorical(y_train, 43)
valid_labels = to_categorical(valid_labels, 43)

In [16]:
print(y_train.shape)
print(valid_labels.shape)

(39209, 43)
(7578, 43)


In [17]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
# We have 43 classes that's why we have defined 43 in the dense
model.add(Dense(43, activation='softmax'))

In [18]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
epochs = 20
history = model.fit(X_train, y_train, batch_size=32, epochs=epochs, validation_data=(valid_features, valid_labels))

Epoch 1/20
1226/1226 [==============================] - 73s 59ms/step - loss: 1.6236 - accuracy: 0.5696 - val_loss: 0.6184 - val_accuracy: 0.8290
Epoch 2/20
1226/1226 [==============================] - 82s 67ms/step - loss: 0.5939 - accuracy: 0.8227 - val_loss: 0.4409 - val_accuracy: 0.8761
Epoch 3/20
1226/1226 [==============================] - 83s 68ms/step - loss: 0.4582 - accuracy: 0.8638 - val_loss: 0.3016 - val_accuracy: 0.9188
Epoch 4/20
1226/1226 [==============================] - 83s 68ms/step - loss: 0.3943 - accuracy: 0.8827 - val_loss: 0.3164 - val_accuracy: 0.9173
Epoch 5/20
1226/1226 [==============================] - 91s 74ms/step - loss: 0.3650 - accuracy: 0.8910 - val_loss: 0.2797 - val_accuracy: 0.9166
Epoch 6/20
1226/1226 [==============================] - 105s 86ms/step - loss: 0.3387 - accuracy: 0.9002 - val_loss: 0.2539 - val_accuracy: 0.9211
Epoch 7/20
1226/1226 [==============================] - 108s 88ms/step - loss: 0.3240 - accuracy: 0.9037 - val_loss: 0.2827

### Accuracy Curve

In [1]:
loss_train = history.history['train_loss']
loss_val = history.history['val_loss']
epochs = range (1, 25)
plt.plot(epochs, loss_train, 'r', label='Training loss')
plt.plot(epochs, loss_val, 'b', label = 'validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

NameError: name 'history' is not defined